In [12]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import locale
import folium
import os

### Comuni ISTAT 2020

In [16]:
gdf_comuni = gpd.read_file(r"../input/comuni_istat_2020_wgs84.shp")[['COMUNE','PRO_COM','geometry']]
gdf_comuni.rename(columns={'PRO_COM':'CodIstat','COMUNE':'Comune'}, inplace=True)
# gdf_comuni.head(3)

### Focolai puliti

importa i focolai già ripuliti dal precedente script

In [17]:
df_focolai = pd.read_excel(r"../output/focolai.xlsx")
df_focolai.head(3)

,AnnoSospetto,DataSospetto,DataConferma,IdFocolaio,CodAzienda,CoordX,CoordY,CodIstat,Regione,Prov,Comune,Specie,Categorie,CodSierotipo,Sierotipo
0,2020,2020-10-29,2020-11-27,48394,014MI007,9.449249,45.588372,15014,LOMBARDIA,MI,BASIANO,GAZZA,UCCELLI TARGET,2,Lineage 2
1,2020,2020-10-25,2020-11-26,48429,AL22122D,8.463033,45.357862,2017,PIEMONTE,VC,BORGO VERCELLI,CORNACCHIA,UCCELLI TARGET,2,Lineage 2
2,2020,2020-09-05,2020-11-26,48039,AL21081D,9.107900,45.275000,18173,LOMBARDIA,PV,VELLEZZO BELLINI,SPARVIERO,UCCELLI SELVATICI,2,Lineage 2


## Aggregazioni (anno 2018)

In [38]:
# EQUIDI
equidi = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo"]].query("DataSospetto >= '2018-01-01' & DataSospetto <= '2018-12-31' & Categorie == 'EQUIDI'")
equidi_aggreg = equidi.groupby(['CodIstat'])["CodIstat"].count().reset_index(name="count")

# AVICOLI
avicoli = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo"]].query("DataSospetto >= '2018-01-01' & DataSospetto <= '2018-12-31' & Categorie == 'AVICOLI'")
avicoli_aggreg = avicoli.groupby(['CodIstat'])["CodIstat"].count().reset_index(name="count")

# UCCELLI SELVATICI
uccelli_selv = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo"]].query("DataSospetto >= '2018-01-01' & DataSospetto <= '2018-12-31' & Categorie == 'UCCELLI SELVATICI'")
uccelli_selv_aggreg = uccelli_selv.groupby(['CodIstat'])["CodIstat"].count().reset_index(name="count")

# UCCELLI TARGET
uccelli_targ = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo"]].query("DataSospetto >= '2018-01-01' & DataSospetto <= '2018-12-31' & Categorie == 'UCCELLI TARGET'")
uccelli_targ_aggreg = uccelli_targ.groupby(['CodIstat'])["CodIstat"].count().reset_index(name="count")

# INSETTI
insetti = df_focolai[["DataSospetto","IdFocolaio","CodIstat","Specie","Categorie","Sierotipo"]].query("DataSospetto >= '2018-01-01' & DataSospetto <= '2018-12-31' & Categorie == 'INSETTI'")
insetti_aggreg = insetti.groupby(['CodIstat'])["CodIstat"].count().reset_index(name="count")

### join con i comuni

In [39]:
equidi_per_comune = gdf_comuni.merge(equidi_aggreg, on='CodIstat')
avicoli_per_comune = gdf_comuni.merge(avicoli_aggreg, on='CodIstat')
uccelli_selv_per_comune = gdf_comuni.merge(uccelli_selv_aggreg, on='CodIstat')
uccelli_targ_per_comune = gdf_comuni.merge(uccelli_targ_aggreg, on='CodIstat')
insetti_per_comune = gdf_comuni.merge(insetti_aggreg, on='CodIstat')

### Mappa aggregazioni per Comune

In [40]:
map = folium.Map([42.783333, 12.866667], zoom_start=5, tiles='openstreetmap')

dist_equidi = folium.features.GeoJson(
    equidi_per_comune.to_json(), 
    name='Equidi', 
    style_function=lambda x: {'fillColor':'#FF0000','fillOpacity': 0.75, 'color':'#CC0000','weight':0.5},
    show=True)

dist_avicoli = folium.features.GeoJson(
    avicoli_per_comune.to_json(), 
    name='Avicoli', 
    style_function=lambda x: {'fillColor':'#FF0000','fillOpacity': 0.75, 'color':'#CC0000','weight':0.5},
    show=False)

dist_ucc_selv = folium.features.GeoJson(
    uccelli_selv_per_comune.to_json(), 
    name='Uccelli Selvatici', 
    style_function=lambda x: {'fillColor':'#FF0000','fillOpacity': 0.75, 'color':'#CC0000','weight':0.5},
    show=False)

dist_ucc_targ = folium.features.GeoJson(
    uccelli_targ_per_comune.to_json(), 
    name='Uccelli Target', 
    style_function=lambda x: {'fillColor':'#FF0000','fillOpacity': 0.75, 'color':'#CC0000','weight':0.5},
    show=False)

dist_insetti = folium.features.GeoJson(
    insetti_per_comune.to_json(), 
    name='Insetti', 
    style_function=lambda x: {'fillColor':'#FF0000','fillOpacity': 0.75, 'color':'#CC0000','weight':0.5},
    show=False)

map.add_child(dist_equidi)
map.add_child(dist_avicoli)
map.add_child(dist_ucc_selv)
map.add_child(dist_ucc_targ)
map.add_child(dist_insetti)

map.add_child(folium.LayerControl())

map